In [6]:
# ! pip install spacy
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.6 MB/s eta 0:00:05
     - -------------------------------------- 0.5/12.8 MB 4.4 MB/s eta 0:00:03
     --- ------------------------------------ 1.3/12.8 MB 8.0 MB/s eta 0:00:02
     ---- ----------------------------------- 1.5/12.8 MB 7.6 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 12.5 MB/s eta 0:00:01
     -------------- ------------------------- 4.7/12.8 MB 14.9 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 15.4 MB/s eta 0:00:01
     -------------------- ------------------- 6.5/12.8 MB 16.0 MB/s eta 0:00:01
     ----------------------- ---------------- 7.5/12.8 MB 16.6 MB/s eta 0:00:01
     -------------------------- ------------- 8.5/12.8 MB 16.9 MB/s eta 0:00:01
     ----------------------------- ---------- 9.5/12.8 MB 17.3 MB/s eta 0:00:01
     ------------------------------- ------- 10.3/12

In [ ]:
# NER

import spacy

nlp = spacy.load("en_core_web_sm")

def has_location(title):
    doc = nlp(title)
    return any(ent.label_ in ("GPE", "LOC") for ent in doc.ents)


# Filter titles that mention a location
location_titles = [title for title in events['title'].dropna().values if has_location(title)]

titles_with_loc = []
print("Titles with locations:")
for t in location_titles:
    print("-", t)
    titles_with_loc.append(t)

Titles with locations:
- HAWAIIAN VOLCANO OBSERVATORY DAILY UPDATE Monday , January 1 , 2024 , 19 : 00 UTC | USGS Hazard Notification System ( HANS ) for Volcanoes


AttributeError: 'list' object has no attribute 'appened'

In [1]:
import pandas as pd

In [2]:
events = pd.read_csv('Nat Cat Events.csv')

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91479 entries, 0 to 91478
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   url            91479 non-null  object
 1   url_mobile     25383 non-null  object
 2   title          91384 non-null  object
 3   seendate       91479 non-null  object
 4   socialimage    79390 non-null  object
 5   domain         91479 non-null  object
 6   language       91479 non-null  object
 7   sourcecountry  89958 non-null  object
dtypes: object(8)
memory usage: 5.6+ MB


In [13]:
import socket
import requests

def get_server_location(url):
    # Strip protocol if present
    if url.startswith("http://") or url.startswith("https://"):
        url = url.split("//")[1].split("/")[0]
    
    try:
        # Get IP address of the domain
        ip_address = socket.gethostbyname(url)
        
        # Use ipinfo.io to get geolocation data
        response = requests.get(f"https://ipinfo.io/{ip_address}/json")
        data = response.json()
        
        location = {
            "IP": ip_address,
            "City": data.get("city"),
            "Region": data.get("region"),
            "Country": data.get("country"),
            "Org": data.get("org"),
            "Loc": data.get("loc")  # Latitude and Longitude
        }
        return location
    except Exception as e:
        return {"error": str(e)}

In [ ]:
import math
url_server_locations = []

for url in events['url'].values[:20]:
    url_server_locations.append(get_server_location(url))

In [24]:
import math
url_mobile_server_locations = []

for url in events['url_mobile'].values[:20]:
    if pd.notna(url):
        url_mobile_server_locations.append(get_server_location(url))

In [6]:
print(location)

{'IP': '151.101.65.91', 'City': 'San Francisco', 'Region': 'California', 'Country': 'US', 'Org': 'AS54113 Fastly, Inc.', 'Loc': '37.7621,-122.3971'}


In [25]:
import folium

def show_location_on_map(map_, server_locations, marker_colour):
    for location_data in server_locations:
        latitude, longitude = location_data['Loc'].split(',')
        folium.Marker((latitude, longitude), icon=folium.Icon(color=marker_colour, icon="server", prefix='fa')).add_to(map_)
    return map_

map_ = folium.Map([20, 0], zoom_start=4)
map_ = show_location_on_map(map_, url_server_locations, "blue")
map_ = show_location_on_map(map_, url_mobile_server_locations, "red")


In [26]:
map_

In [ ]:
events.describe()

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
count,91479,25383,91384,91479,79390,91479,91479,89958
unique,89303,24501,65158,23354,63431,5878,1,135
top,https://news.webindia123.com/news/Articles/Wor...,https://www.aljazeera.com/amp/news/liveblog/20...,2 . 9 Magnitude Earthquake Reported In US,20240829T073000Z,https://i.iheart.com/v3/re/new_assets/6653968c...,yahoo.com,English,United States
freq,2,23,212,71,236,4691,91479,59232


In [5]:
events['seendate']

0        20240101T223000Z
1        20240101T220000Z
2        20240101T124500Z
3        20240101T223000Z
4        20240101T090000Z
               ...       
91474    20241231T061500Z
91475    20241231T073000Z
91476    20241231T091500Z
91477    20241231T174500Z
91478    20241231T011500Z
Name: seendate, Length: 91479, dtype: object